In [2]:
import getpass
import os


def _set_env(var: str):
    env_exist=os.environ.get(var)
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")
    else:
        print(f"env existist: {env_exist}")

_set_env("OPENAI_API_KEY")

env existist: REMOVED_SECRET


In [6]:
import json
try:
    with open("nginx_metrics_compact.json", "r") as compact_file:
        existing_summary = json.load(compact_file)

except FileNotFoundError:
    raise FileNotFoundError("nginx_metrics_compact.json not found")
# LLM prompt
prompt = f"""
Analyze the following NGINX ingress metrics:

Metrics Summary:
{json.dumps(existing_summary, indent=2)}

Based on the patterns over the last 5m, 1h, 3h, 6h, 12h, 1d, 3d, and 7d, recommend if the load balancing algorithm should be changed. Provide reasons for your recommendation.
"""


In [7]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # api_key="...",
    # base_url="...",
    # organization="...",
    # other params...
)

In [8]:
messages = [
    (
        "system",
        """ Analyze the following NGINX ingress metrics and recommend if the load balancing algorithm should be changed. 
            Provide clear reasons for your recommendation based on the metrics patterns."""
    ),
    ("human", prompt),
]
llm.invoke(messages)

AIMessage(content='Based on the provided NGINX ingress metrics, it appears that there is no traffic being processed by the system. The request count is consistently zero across all time intervals (5m, 1h, 3h, 6h, 12h, 1d, 3d, and 7d). Additionally, the average response time and the 95th percentile response time are also zero or NaN, indicating no requests are being handled. There are no active or dropped connections recorded in any of the time intervals.\n\nGiven this data, there is no evidence of load or traffic that would necessitate a change in the load balancing algorithm. The current metrics suggest that the system is either not receiving any traffic or is not correctly capturing the traffic data. \n\nHere are the reasons for not recommending a change in the load balancing algorithm:\n\n1. **No Traffic Detected**: The request count is zero across all time frames, indicating that there is no load to balance. Without traffic, the effectiveness of the load balancing algorithm cannot 

In [20]:
import operator
from typing import Annotated, List, Tuple
from typing_extensions import TypedDict


class PlanExecute(TypedDict):
    input: str
    plan: List[str]
    past_steps: Annotated[List[Tuple], operator.add]
    response: str


from pydantic import BaseModel, Field


class Plan(BaseModel):
    """Plan to follow in future"""

    steps: List[str] = Field(
        description="different steps to follow, should be in sorted order"
    )

In [21]:
from langchain_core.prompts import ChatPromptTemplate

planner_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """For the given objective, come up with a simple step by step plan. \
This plan should involve individual tasks, that if executed correctly will yield the correct answer. Do not add any superfluous steps. \
The result of the final step should be the final answer. Make sure that each step has all the information needed - do not skip steps.""",
        ),
        ("placeholder", "{messages}"),
    ]
)
planner = planner_prompt | ChatOpenAI(
    model="gpt-4o", temperature=0
).with_structured_output(Plan)

In [23]:
planner.invoke(
    {
        "messages": [
            ("user", "What should be my plans for next year ?")
        ]
    }
)

Plan(steps=['Identify your personal and professional goals for the next year.', 'Break down each goal into smaller, manageable tasks.', 'Set a timeline for each task, ensuring they are realistic and achievable.', 'Prioritize tasks based on importance and deadlines.', 'Allocate resources and time for each task.', 'Monitor progress regularly and adjust the plan as needed.', 'Celebrate achievements and learn from any setbacks.'])